# Project 01 - Detecting the Laboratory Environment

Use the "Grove Starter Kit for Raspberry Pi Pico" to build an environmental temperature/humidity/light sensor. The sensor should

* Use the DHT11 module to measure temperature and humidity of the laboraatory environment.
* Use the Grove light sensor v1.2 to measure light intensity.
* Report current values on the Grive LCD1602 display.

Use an update period of 1.0 seconds.

## Installations

This project requires download of several Python libraries to the Raspberry Pi Pico. The necessary libraries, additional documentation, and examples are available as package for files from [Beginners Guide of Raspberry Pi Pico Based on MicroPython](https://files.seeedstudio.com/Seeed_EDU/Course_documents/Beginner's-Guide-for-Raspberry-Pi-Pico.zip) available from Seeed Studio. Click the link to download the package to a convenient location on your laptop, unzip package, then locate and open the pdf file "Beginner's Guide for Raspberry Pi Pico".

## Verify Communicaton with the Raspberry Pi Pico

https://github.com/goatchurchprime/jupyter_micropython_kernel

In [11]:
%serialconnect

Found serial ports: /dev/cu.usbmodem141101, /dev/cu.BLTH, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem141101 --baud=115200 
O[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

The following code provides a simple check that MicroPython code is successfully downloaded and executes on a connected Raspberry Pi Pico. 

In [4]:
import machine

led = machine.Pin(25, machine.Pin.OUT)

def toggle_led(timer):
    led.toggle()

timer = machine.Timer(freq=20, mode=machine.Timer.PERIODIC, callback=toggle_led)

Run the following code to deinitialize the timer and turn off the on-board led.

In [5]:
timer.deinit()
led.value(0)

**Exercise:** Create a script to blink the led for a period of 10 seconds, then turn the blinker off and return the led to an off state.

In [2]:
import machine
import time

led = machine.Pin(25, machine.Pin.OUT)

def toggle_led(timer):
    led.toggle()

timer = machine.Timer(freq=20, mode=machine.Timer.PERIODIC, callback=toggle_led)
time.sleep(10)
timer.deinit()

led.value(0)

.

## Light Sensor

The light sensor requires as an analog to digital conversion. Analog to digital conversion is the task of measuring an analog signal and converting it to a digital measurement. 

https://datasheets.raspberrypi.com/pico/pico-datasheet.pdf

The Raspberry Pi Pico includes four independent ADC channels, of which three are available for external use on GPIO pins GP26, GP27, and GP28. The pins measure a voltage by comparing the signal to the power supply of 3.3 volts and producing a 12 bit integer value between 0 and 4095 with a sampling rate of approximately 500 kHz.

Micropython provides an interface to the Raspberry Pi Pico's ADC channels as documented below.

https://docs.micropython.org/en/latest/library/machine.ADC.html

**Exercise:** Write a python code to read the light sensor at a frequency of 10 Hz, and print light intensity as a voltage.

In [4]:
import machine
import time

adc = machine.ADC(0)
def print_adc(timer):
    val = adc.read_u16()
    print(val/(2**16 - 1))
    
timer = machine.Timer(freq=10, mode=machine.Timer.PERIODIC, callback=print_adc)
time.sleep(10)
timer.deinit()

0.6024414
0.6031738
0.6024414
0.6095216
0.6078126
0.6029297
0.6031738
0.6029297
0.6039063
0.6097658
0.6078126
0.6043946
0.6065919
0.6107423
0.6056153
0.6056153
0.6056153
0.6043946
0.6058595
0.6104982
0.605127
0.6041505
0.6075685
0.6002441
0.6087892
0.606836
0.6043946
0.6080568
0.6056153
0.6026855
0.6080568
0.6080568
0.6124514
0.6090333
0.6021973
0.6021973
0.6056153
0.610254
0.6031738
0.6039063
0.6036622
0.6041505
0.6100099
0.610254
0.6065919
0.6048829
0.6012207
0.601709
0.6100099
0.6024414
0.6083009
0.6012207
0.6039063
0.6031738
0.6100099
0.601709
0.6029297
0.6085451
0.6095216
0.601709
0.6053712
0.6039063
0.6075685
0.6041505
0.6029297
0.6070802
0.6041505
0.605127
0.6036622
0.6031738
0.6043946
0.6109865
0.6036622
0.606836
0.6141603
0.6061036
0.6104982
0.6029297
0.6065919
0.6061036
0.6065919
0.6063478
0.6046388
0.6070802
0.6021973
0.6129396
0.6043946
0.6056153
0.6031738
0.6029297
0.6192874
0.6100099
0.6039063
0.6046388
0.6031738
0.6043946
0.605127
0.6097658
0.6085451
0.6100099


## Display Output on LCD1602

In [7]:
import machine
import time
from lcd1602 import LCD1602

class LCD16x2():
    
    def __init__(self, i2c):
        self.i2c = i2c
        self.d = LCD1602(i2c, 2, 16)
        self.d.clear()
        self.lines = [" "*16, " "*16]
        
    def update(self, line1=None, line2=None):
        self.update_line(0, line1)
        self.update_line(1, line2)
        
    def update_line(self, j, line):
        line = "{:16s}".format(line)
        if line != self.lines[j]:
            for i, char in enumerate(line):
                self.d.setCursor(i, j)
                self.d.write(ord(char))
            self.lines[j] = line

def display_adc(timer):
    global lcd
    global adc
    value = adc.read_u16()
    lcd.update(f"{value}", "")
    
# specify i2c
sda = machine.Pin(8, machine.Pin.OUT)
scl = machine.Pin(9, machine.Pin.OUT)
i2c = machine.I2C(0, sda=sda, scl=scl)

# create lcd instance
lcd = LCD16x2(i2c)
adc = machine.ADC(0)


# update using Timer
timer = machine.Timer(freq=10, mode=machine.Timer.PERIODIC, callback=display_adc)
time.sleep(5)
timer.deinit()

.

## Temperature and Humidity Sensor v1.2

Data Sheet

https://www.seeedstudio.com/Grove-Temperature-Humidity-Sensor-DHT11.html

In [14]:
import machine
import time
from lcd1602 import LCD1602
import dht11

class LCD16x2():
    
    def __init__(self, i2c):
        self.i2c = i2c
        self.d = LCD1602(i2c, 2, 16)
        self.d.clear()
        self.lines = [" "*16, " "*16]
        
    def update(self, line1=None, line2=None):
        self.update_line(0, line1)
        self.update_line(1, line2)
        
    def update_line(self, j, line):
        line = "{:16s}".format(line)
        if line != self.lines[j]:
            for i, char in enumerate(line):
                self.d.setCursor(i, j)
                self.d.write(ord(char))
            self.lines[j] = line

def display_data(timer):
    global lcd
    global adc
    global dht
    light = adc.read_u16()
    temp, humid = dht.readTempHumid()
    print(temp, humid)
    lcd.update(f"light = {light}", f"T = {temp}  H={humid}")
    
# specify i2c
sda = machine.Pin(8, machine.Pin.OUT)
scl = machine.Pin(9, machine.Pin.OUT)
i2c = machine.I2C(0, sda=sda, scl=scl)

# create lcd instance
lcd = LCD16x2(i2c)
adc = machine.ADC(0)
dht = dht11.DHT(18)


# update using Timer
timer = machine.Timer(freq=10, mode=machine.Timer.PERIODIC, callback=display_data)
time.sleep(5)
timer.deinit()


No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [7]:
timer.deinit()